# Finetuning falcon-1b with Axolotl+QLoRA

This notebook makes it easy to try out finetuning falcon-1b with Axolotl+QLoRA.

axolotl link [here](https://github.com/OpenAccess-AI-Collective/axolotl) .

## 1. Get Axolotl and install its dependencies

In [1]:
!git clone https://github.com/OpenAccess-AI-Collective/axolotl.git

Cloning into 'axolotl'...
remote: Enumerating objects: 7534, done.
remote: Counting objects: 100% (1735/1735), done.
remote: Compressing objects: 100% (394/394), done.
remote: Total 7534 (delta 1532), reused 1417 (delta 1318), pack-reused 5799
Receiving objects: 100% (7534/7534), 2.64 MiB | 6.62 MiB/s, done.
Resolving deltas: 100% (4861/4861), done.


In [2]:
%cd axolotl

/content/axolotl


In [3]:
%%capture
!pip3 install -e .

## 2. Generate default config for accelerate



In [6]:
!accelerate config default

accelerate configuration saved at /root/.cache/huggingface/accelerate/default_config.yaml


In [7]:
!cat /root/.cache/huggingface/accelerate/default_config.yaml

{
  "compute_environment": "LOCAL_MACHINE",
  "debug": false,
  "distributed_type": "NO",
  "downcast_bf16": false,
  "machine_rank": 0,
  "main_training_function": "main",
  "mixed_precision": "no",
  "num_machines": 1,
  "num_processes": 1,
  "rdzv_backend": "static",
  "same_network": false,
  "tpu_use_cluster": false,
  "tpu_use_sudo": false,
  "use_cpu": false
}


## 3. Use a well-tested falcon-7b qlora config and adjust it to 1b & Cloab

In [4]:
!wget https://raw.githubusercontent.com/sam-ai/axolotl/main/examples/falcon/config-1b-qlora.yml -O /content/axolotl/examples/falcon/config-1b-qlora.yml

--2023-11-23 04:34:56--  https://raw.githubusercontent.com/sam-ai/axolotl/main/examples/falcon/config-1b-qlora.yml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2228 (2.2K) [text/plain]
Saving to: ‘/content/axolotl/examples/falcon/config-1b-qlora.yml’

/content/axolotl/ex 100%[===================>]   2.18K  --.-KB/s    in 0s      

2023-11-23 04:34:56 (34.3 MB/s) - ‘/content/axolotl/examples/falcon/config-1b-qlora.yml’ saved [2228/2228]



In [ ]:
# add hub_model_id: <hugginface-repo-path> in config file

In [5]:
!cat /content/axolotl/examples/falcon/config-1b-qlora.yml

base_model: tiiuae/falcon-rw-1b
base_model_config: tiiuae/falcon-rw-1b
# required by falcon custom model code: https://huggingface.co/tiiuae/falcon-7b/tree/main
trust_remote_code: true
model_type: AutoModelForCausalLM
tokenizer_type: AutoTokenizer
is_falcon_derived_model: true
load_in_8bit: false
# enable 4bit for QLoRA
load_in_4bit: true
gptq: false
strict: false
push_dataset_to_hub:
datasets:
  - path: teknium/GPT4-LLM-Cleaned
    type: alpaca:instruct
dataset_prepared_path: last_run_prepared
val_set_size: 0.01
# enable QLoRA
adapter: qlora
lora_model_dir:
sequence_len: 2048
max_packed_sequence_len:

# hyperparameters from QLoRA paper Appendix B.2
# "We find hyperparameters to be largely robust across datasets"
lora_r: 64
lora_alpha: 16
# 0.1 for models up to 13B
# 0.05 for 33B and 65B models
lora_dropout: 0.05
# add LoRA modules on all linear layers of the base model
lora_target_modules:
lora_target_linear: true
lora_fan_in_fan_out:

# lora_modules_to_save:
#   - word_embeddings
#  

You can read the comments in the config and tweak it.

## 4. Install QLoRA dependencies

In [8]:
%%capture
!pip install bitsandbytes
!pip install git+https://github.com/huggingface/transformers.git
!pip install git+https://github.com/huggingface/peft.git
!pip install git+https://github.com/huggingface/accelerate.git

## 5. Set W&B to offline mode

In [9]:
%env WANDB_MODE=online

env: WANDB_MODE=offline


This is to skip some extra setup steps, you can also choose to login to your W&B account  before training.


## 6. Start training and enjoy!

In [10]:
!accelerate launch -m axolotl.cli.train /content/axolotl/examples/falcon/config-1b-qlora.yml

2023-11-23 04:40:38.597349: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-23 04:40:38.597423: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-23 04:40:38.597461: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-23 04:40:40.681662: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrati

In [ ]:
# https://colab.research.google.com/drive/1AfE71MfayK4qo9DxdTR5oAeJdtZtVtyG#offline=true&sandboxMode=true